# <center>
<div style="
    background: #f0f333ff;
    border-left: 5px solid #ecd242ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h1 style="
        text-align: center;
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 600;
    ">
    My Scheduler
    </h1>
</div>

##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    CA Scheduling
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    First example : From chatGPT
    </h3>
</div>

In [39]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Paramètres du problème
    # -------------------------
    employees = ["A", "B", "C", "D"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    shifts = ["Matin", "Après-midi"]
    activities = ["Telephone", "Derogation", "Reclamation"]

    n_emp = len(employees)
    n_days = len(days)
    n_shifts = len(shifts)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for s in range(n_shifts):
                for a in range(len(activities)):
                    x[e, d, s, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{shifts[s]}_{activities[a]}")

    # -------------------------
    # Contraintes
    # -------------------------

    # 1️⃣ Chaque créneau : 2 Téléphones, 1 Dérogation, 1 Réclamation
    for d in range(n_days):
        for s in range(n_shifts):
    #         # Chaque activité a le bon nombre de personnes
            model.Add(sum(x[e, d, s, 0] for e in range(n_emp)) == 2)  # Téléphone
            model.Add(sum(x[e, d, s, 1] for e in range(n_emp)) == 1)  # Dérogation
            model.Add(sum(x[e, d, s, 2] for e in range(n_emp)) == 1)  # Réclamation

            # Chaque employé fait exactement 1 activité par créneau
            for e in range(n_emp):
                model.Add(sum(x[e, d, s, a] for a in range(len(activities))) == 1)

    # 2️⃣ Pas plus de 3 jours de Téléphone par semaine par employé
    # Un jour est compté Téléphone s'il fait téléphone matin ou après-midi
    for e in range(n_emp):
        is_tel_day = []
        for d in range(n_days):
            tel_day = model.NewBoolVar(f"tel_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) >= 1).OnlyEnforceIf(tel_day)
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) == 0).OnlyEnforceIf(tel_day.Not())
            is_tel_day.append(tel_day)
        model.Add(sum(is_tel_day) <= 3)

    # 3️⃣ Pas 3 jours consécutifs de Téléphone
    for e in range(n_emp):
        for d in range(n_days - 2):
            model.Add(sum(x[e, d + i, s, 0] for i in range(3) for s in range(n_shifts)) <= 4)
            # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    solver.parameters.num_search_workers = 8

    result = solver.Solve(model)

    # -------------------------
    # Affichage du planning
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for d in range(n_days):
            print(f"\n=== {days[d]} ===")
            for s in range(n_shifts):
                print(f"  {shifts[s]} :")
                for e in range(n_emp):
                    for a in range(len(activities)):
                        if solver.Value(x[e, d, s, a]) == 1:
                            print(f"    {employees[e]} → {activities[a]}")
    else:
        print("❌ Aucune solution trouvée.")

if __name__ == "__main__":
    main()


=== Lundi ===
  Matin :
    A → Derogation
    B → Telephone
    C → Reclamation
    D → Telephone
  Après-midi :
    A → Derogation
    B → Telephone
    C → Reclamation
    D → Telephone

=== Mardi ===
  Matin :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone
  Après-midi :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone

=== Mercredi ===
  Matin :
    A → Telephone
    B → Telephone
    C → Derogation
    D → Reclamation
  Après-midi :
    A → Telephone
    B → Telephone
    C → Reclamation
    D → Derogation

=== Jeudi ===
  Matin :
    A → Derogation
    B → Reclamation
    C → Telephone
    D → Telephone
  Après-midi :
    A → Telephone
    B → Derogation
    C → Telephone
    D → Reclamation

=== Vendredi ===
  Matin :
    A → Telephone
    B → Telephone
    C → Reclamation
    D → Derogation
  Après-midi :
    A → Derogation
    B → Telephone
    C → Telephone
    D → Reclamation


**My custom**

Tu veux donc un planning hebdomadaire (5 jours) avec :

4 employés (nommons-les : A, B, C, D)

2 créneaux par jour (matin, après-midi)

3 activités : téléphone, dérogation, réclamation

Contraintes :

Chaque créneau (matin, après-midi) → 2 Téléphones, 1 Dérogation, 1 Réclamation

Un employé ne fait pas plus de 3 jours de Téléphone par semaine.

Un employé ne fait pas 3 jours consécutifs de Téléphone (le matin et/ou l’après-midi comptent comme « du téléphone »).

In [59]:
from ortools.sat.python import cp_model

def main():
    # -------------------------
    # Paramètres du problème
    # -------------------------
    employees = ["A", "B", "C", "D"]
    days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    shifts = ["Matin", "Après-midi"]
    activities = ["Tél", "Dérog", "Récla"]

    n_emp = len(employees)
    n_days = len(days)
    n_shifts = len(shifts)

    # -------------------------
    # Modèle
    # -------------------------
    model = cp_model.CpModel()

    # x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
    x = {}
    for e in range(n_emp):
        for d in range(n_days):
            for s in range(n_shifts):
                for a in range(len(activities)):
                    x[e, d, s, a] = model.NewBoolVar(f"x_{employees[e]}_{days[d]}_{shifts[s]}_{activities[a]}")

    # -------------------------
    # Contraintes
    # -------------------------

    # 1️⃣ Chaque créneau : 2 Téléphones, 1 Dérogation, 1 Réclamation
    for d in range(n_days):
        for s in range(n_shifts):
            # Chaque activité a le bon nombre de personnes
            model.Add(sum(x[e, d, s, 0] for e in range(n_emp)) == 2)  # Téléphone
            model.Add(sum(x[e, d, s, 1] for e in range(n_emp)) == 1)  # Dérogation
            model.Add(sum(x[e, d, s, 2] for e in range(n_emp)) == 1)  # Réclamation

            # Chaque employé fait exactement 1 activité par créneau
            for e in range(n_emp):
                model.Add(sum(x[e, d, s, a] for a in range(len(activities))) == 1)

    # 2️⃣ Pas plus de 3 jours de Téléphone par semaine par employé
    # Un jour est compté Téléphone s'il fait téléphone matin ou après-midi
    for e in range(n_emp):
        is_tel_day = []
        for d in range(n_days):
            tel_day = model.NewBoolVar(f"tel_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) >= 1).OnlyEnforceIf(tel_day)
            model.Add(sum(x[e, d, s, 0] for s in range(n_shifts)) == 0).OnlyEnforceIf(tel_day.Not())
            is_tel_day.append(tel_day)
        model.Add(sum(is_tel_day) <= 3)

    # 3️⃣ Pas 3 jours consécutifs de Téléphone
    for e in range(n_emp):
        for d in range(n_days - 2):
            model.Add(sum(x[e, d + i, s, 0] for i in range(3) for s in range(n_shifts)) <= 4)
            # (au max 4 créneaux sur 6 possibles pour 3 jours → évite 3 jours pleins de téléphone)

    # 4️⃣ Pas 2 jours consécutifs de Dérogation
    for e in range(n_emp):
        for d in range(n_days - 1):
            # Jour d : il fait dérogation ?
            derog_d = model.NewBoolVar(f"derog_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 1] for s in range(n_shifts)) >= 1).OnlyEnforceIf(derog_d)
            model.Add(sum(x[e, d, s, 1] for s in range(n_shifts)) == 0).OnlyEnforceIf(derog_d.Not())

            # Jour suivant : dérogation ?
            derog_next = model.NewBoolVar(f"derog_day_{employees[e]}_{days[d+1]}")
            model.Add(sum(x[e, d+1, s, 1] for s in range(n_shifts)) >= 1).OnlyEnforceIf(derog_next)
            model.Add(sum(x[e, d+1, s, 1] for s in range(n_shifts)) == 0).OnlyEnforceIf(derog_next.Not())

            # Pas deux jours consécutifs de dérogation
            model.AddBoolOr([derog_d.Not(), derog_next.Not()])

    # 5️⃣ Pas 2 jours consécutifs de Réclamation
    for e in range(n_emp):
        for d in range(n_days - 1):
            # Jour d : fait-il réclamation ?
            recl_d = model.NewBoolVar(f"reclam_day_{employees[e]}_{days[d]}")
            model.Add(sum(x[e, d, s, 2] for s in range(n_shifts)) >= 1).OnlyEnforceIf(recl_d)
            model.Add(sum(x[e, d, s, 2] for s in range(n_shifts)) == 0).OnlyEnforceIf(recl_d.Not())

            # Jour suivant : fait-il réclamation ?
            recl_next = model.NewBoolVar(f"reclam_day_{employees[e]}_{days[d+1]}")
            model.Add(sum(x[e, d+1, s, 2] for s in range(n_shifts)) >= 1).OnlyEnforceIf(recl_next)
            model.Add(sum(x[e, d+1, s, 2] for s in range(n_shifts)) == 0).OnlyEnforceIf(recl_next.Not())

            # Pas deux jours consécutifs de réclamation
            model.AddBoolOr([recl_d.Not(), recl_next.Not()])

    # 6️⃣ Objectif : minimiser les changements d'activité entre matin et après-midi
    penalties = []
    for e in range(n_emp):
        for d in range(n_days):
            # Variable binaire : 1 si les activités matin et après-midi sont différentes
            different_activity = model.NewBoolVar(f"different_{employees[e]}_{days[d]}")

            # Pour chaque activité, on vérifie si le matin et l'après-midi correspondent
            same_activity_bools = []
            for a in range(len(activities)):
                both_same = model.NewBoolVar(f"same_{employees[e]}_{days[d]}_{activities[a]}")
                model.AddBoolAnd([x[e, d, 0, a], x[e, d, 1, a]]).OnlyEnforceIf(both_same)
                model.AddBoolOr([x[e, d, 0, a].Not(), x[e, d, 1, a].Not()]).OnlyEnforceIf(both_same.Not())
                same_activity_bools.append(both_same)

            # Si au moins une activité est identique sur la journée, alors different_activity = 0
            model.AddBoolOr(same_activity_bools).OnlyEnforceIf(different_activity.Not())
            # Sinon, different_activity = 1
            model.AddBoolAnd([b.Not() for b in same_activity_bools]).OnlyEnforceIf(different_activity)

            penalties.append(different_activity)

    # On minimise le nombre total de changements d'activité sur la semaine
    model.Minimize(sum(penalties))
    
    # -------------------------
    # Solveur
    # -------------------------
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 10
    solver.parameters.num_search_workers = 8

    result = solver.Solve(model)

    # -------------------------
    # Affichage du planning
    # -------------------------
    if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
        for e in range(n_emp):
            st = f"{employees[e]}  →  "
            for d in range(n_days):
                for s in range(n_shifts):
                    for a in range(len(activities)):
                        if solver.Value(x[e, d, s, a]) == 1:
                            st += f"{activities[a]:6} "
            print(st)
    else:
        print("❌ Aucune solution trouvée.")

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")

if __name__ == "__main__":
    main()


A  →  Récla  Récla  Dérog  Dérog  Récla  Récla  Tél    Tél    Tél    Tél    
B  →  Tél    Tél    Récla  Récla  Tél    Tél    Tél    Tél    Dérog  Dérog  
C  →  Dérog  Dérog  Tél    Tél    Tél    Tél    Dérog  Dérog  Récla  Récla  
D  →  Tél    Tél    Tél    Tél    Dérog  Dérog  Récla  Récla  Tél    Tél    

Statistics
  - conflicts      : 0
  - branches       : 310
  - wall time      : 0.0237902 s


###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Real Test
    </h3>
</div>

In [ ]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, employees, postes, days, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._employees = employees
        self._postes = postes
        self._days = days
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        print("\t  Poste1   Poste2")
        for d in self._days:
            st = f"Day {d}  →  "
            for p in self._postes:
                for e in self._employees:
                    if self.value(self._shifts[(e, d, p)]):
                        st += f"{e:7}  "
            print(st)
        print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count


# Data.
n_employees = 15
employees = [chr(i) for i in range(65, 65 + n_employees)]
activities = ["Tél", "Rens", "Dérog", "Récla", "Imp", "Libre"]
days = range(10)

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(e, d, p)]: employee 'e' works day 'd' on poste 'p'.
shifts = {}
for e in employees:
    for d in days:
        for p in postes:
            shifts[(e, d, p)] = model.new_bool_var(f"shift_{e}_{d}_{p}")

# Each post must be assigned to exactly one employee each day.
for d in days:
    for p in postes:
        model.add_exactly_one(shifts[(e, d, p)] for e in employees)

# Each employee works at most one post per day.
for d in days:
    for e in employees:
        model.add_at_most_one(shifts[(e, d, p)] for p in postes)

# Each employee works at least one time on each post over the 3 days.
for p in postes:
    for e in employees:
        model.add_at_least_one(shifts[(e, d, p)] for d in days)

# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first five solutions.
solution_limit = 3
solution_printer = PartialSolutionPrinter(shifts, employees, postes, days, solution_limit)
solver.solve(model, solution_printer)

In [66]:
[chr(i) for i in range(65, 90)]

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y']